In [1]:
import numpy as np
import pandas as pd
import glob
import os
import tensorflow as tf
import transformers
from transformers import TFBertForTokenClassification, TFXLMRobertaForTokenClassification
from tqdm.notebook import tqdm
import IPython

import sys
sys.path.append("..")
from data_preparation.data_preparation_pos import MBERT_Tokenizer, XLMR_Tokenizer, convert_examples_to_tf_dataset, read_conll
import utils.utils as utils
import utils.pos_utils as pos_utils

### Training language setup

In [2]:
data_dir = "../data/ud/"
results_path = "../results/xlm_roberta/results_pos.xlsx"
full_model_name = "jplu/tf-xlm-roberta-base"
#full_model_name = "bert-base-multilingual-cased"
model_name = full_model_name.split("/")[-1] # The "jplu/" part is problematic

code_dicts = utils.make_lang_code_dicts()
code_to_name = code_dicts["code_to_name"]
name_to_code = code_dicts["name_to_code"]

# Look for languages that have PoS weights but are not in the results file
file = open("../data_exploration/pos_table.txt", "r")
all_langs = [line.split("&")[1].strip() for line in file.readlines()]
trained_langs = [code_to_name[x.split("\\")[1]] for x in glob.glob("E:/TFM_CCIL/checkpoints/*/{}_pos.hdf5".format(model_name))]
untrained_langs = []
cannot_train_langs = []
for lang in all_langs:
    # Check if there are train and dev sets available
    if (glob.glob(data_dir + name_to_code[lang] + "/*train.conllu") and 
        glob.glob(data_dir + name_to_code[lang] + "/*dev.conllu")):
        if lang not in trained_langs:
            untrained_langs.append(lang)
    else:
        cannot_train_langs.append(lang)

if os.path.isfile(results_path):
    results = pd.read_excel(results_path, sheet_name=None)
    remaining_langs = [lang for lang in trained_langs if lang not in results["Accuracy"].columns]
else:
    remaining_langs = trained_langs
    
evaluated_langs = [lang for lang in trained_langs if lang not in remaining_langs]
    
if remaining_langs:
    training_lang = remaining_langs[0]
    print("Evaluating with:   ", training_lang, "\n")
    training_lang = name_to_code[training_lang]
    print(IPython.utils.text.columnize(["Already evaluated:"] + evaluated_langs, displaywidth=150))
    print(IPython.utils.text.columnize(["Not yet evaluated:"] + remaining_langs[1:], displaywidth=150))
    print(IPython.utils.text.columnize(["Still to train:   "] + untrained_langs, displaywidth=150))
    print(IPython.utils.text.columnize(["Cannot train:     "] + cannot_train_langs, displaywidth=150))
else:
    print("No languages remaining", "\n")
    print(IPython.utils.text.columnize(["Already evaluated:"] + evaluated_langs, displaywidth=150))
    print(IPython.utils.text.columnize(["Still to train:   "] + untrained_langs, displaywidth=150))
    print(IPython.utils.text.columnize(["Cannot train:     "] + cannot_train_langs, displaywidth=150))

Evaluating with:    Arabic 

Already evaluated:

Not yet evaluated:  Bulgarian  English  Basque  Finnish  Hebrew  Croatian  Japanese  Korean  Russian  Slovak  Turkish  Vietnamese  Chinese
Arabic              Bulgarian  English  Basque  Finnish  Hebrew  Croatian  Japanese  Korean  Russian  Slovak  Turkish  Vietnamese  Chinese

Still to train:   

Cannot train:       Thai



### Model setup

In [3]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(gpu_devices[0], True)

In [4]:
# Model parameters
max_length = 256
batch_size = 256
tagset = ["O", "_", "ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", 
          "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]
num_labels = len(tagset)
label_map = {label: i for i, label in enumerate(tagset)}

# Model creation and loading weights
if model_name.startswith("bert"):
    tokenizer = MBERT_Tokenizer.from_pretrained(full_model_name)
    config = transformers.BertConfig.from_pretrained(full_model_name, num_labels=num_labels)
    model = TFBertForTokenClassification.from_pretrained(full_model_name,
                                                         config=config)
else:
    tokenizer = XLMR_Tokenizer.from_pretrained(full_model_name)
    model = TFXLMRobertaForTokenClassification.from_pretrained(full_model_name, num_labels=num_labels)
weights_path = "E:/TFM_CCIL/checkpoints/" + training_lang + "/"
weights_filename = model_name + "_pos.hdf5"
model.load_weights(weights_path + weights_filename)
print("Using weights from", weights_path + weights_filename)

Some weights of the model checkpoint at jplu/tf-xlm-roberta-base were not used when initializing TFXLMRobertaForTokenClassification: ['lm_head']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFXLMRobertaForTokenClassification were not initialized from the model checkpoint at jplu/tf-xlm-roberta-base and are newly initialized: ['dropout_38', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using weights from E:/TFM_CCIL/checkpoints/ar/tf-xlm-roberta-base_pos.hdf5


### Evaluation

In [5]:
pos_eval = []

for directory in tqdm(os.listdir(data_dir)):
    # Load and preprocess
    path = os.path.join(data_dir, directory)
    test_examples, test_dataset = pos_utils.load_data(path, batch_size, tokenizer, tagset, max_length)
    
    # Predict
    preds = model.predict(test_dataset, steps=np.ceil(len(test_examples) / batch_size), verbose=1)
    
    # Postprocessing
    tokens, labels, filtered_preds, logits, subword_locations = pos_utils.filter_padding_tokens(test_examples, preds, 
                                                                                                label_map, tokenizer)
    new_tokens, new_labels, new_preds = pos_utils.reconstruct_subwords(subword_locations, tokens, labels, 
                                                                       filtered_preds, logits)
    
    # Metrics
    accuracy = (np.array(new_labels) == np.array(new_preds)).mean()
    pos_eval.append((directory, accuracy))

4/4 [==============================] - 13s 3s/step



Build the table for this training language

In [6]:
pos_eval = np.array(pos_eval, dtype=object)
table = pd.DataFrame({"Language": pos_eval[:,0],
                      "Accuracy": pos_eval[:,1]})
table["Language"] = table["Language"].apply(lambda x: code_to_name[x])

Reorder so that language types are grouped

In [7]:
file = open("../data_exploration/pos_table.txt", "r")
lang_order = [line.split("&")[1].strip() for line in file.readlines()]
table["sort"] = table["Language"].apply(lambda x: lang_order.index(x))
table = table.sort_values(by=["sort"]).drop("sort", axis=1).reset_index(drop=True)

In [8]:
table

,Language,Accuracy
0,Bulgarian,0.77086
1,English,0.65968
2,Russian,0.762129
3,Slovak,0.771032
4,Croatian,0.762861
5,Chinese,0.460939
6,Vietnamese,0.616646
7,Thai,0.612445
8,Finnish,0.683056
9,Basque,0.59945


Update results excel file

In [9]:
if os.path.isfile(results_path):
    results = pd.read_excel(results_path, sheet_name=None)
else:
    results = dict.fromkeys(table.columns[1:].values, pd.DataFrame({"Language": table["Language"].values}))

In [10]:
with pd.ExcelWriter(results_path) as writer:
    full_training_lang = code_to_name[training_lang]
    for sheet_name, df in results.items():
        # Add each the column for each metric in the corresponding sheet
        df[full_training_lang] = table[sheet_name]
        df.to_excel(writer, index=False, sheet_name=sheet_name)